In [ ]:
# coding=utf-8
# Copyright 2023 Frank Latos AC8P
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Wire Antenna Basics
Design, Visualization, Simulation

## Specifying a wire antenna design
In the following pages we're going to specify wire antenna designs using lists of numpy arrays,
one row per endpoint:
```
my_ant = [np.array([[x0,y0,z0], 
                    [x1,y1,z1], 
                    [x2,y2,z2], 
                     ..... ])]
```
Most designs will need only one array of serially-connected segments, but some (see the cage dipole below) will require a list of several arrays.

We're adopting the convention that antenna elements run parallel to the y axis, and that NEC will mirror elements across the x-z plane, so a simple 40 meter dipole at an elevation of 20 meters would be specified as:
```
my_40m_dipole = [np.array([[0,0,20], [0,10,20]])]
```